# Creating the consumption dataset
For water ticket # WAT-115: Water Use Survey

Will requested that I provide the volume in gallons per connection type per year in addition to the counts I provided previously.

*See [Ticket WAT-115](https://georgetowntx.atlassian.net/jira/servicedesk/projects/WAT/queues/custom/378/WAT-115)*

In [1]:
# Dependencies
import datetime
import gc
import pandas as pd
from src.pymax import umax2012

## Querying data from UMAX 2012

First, we need to collect data from UMAX 2012 since the 2023 billing/consumption data is now split between two databases...

I have developed a custom python package for working with our UMAX CIS called pymax that is used in this notebook.

### Creating the 2012 bill line fact dataset

In this section, I have used the pymax package to extract 2023 bill line data for all water bills with a volumetric unit.

This code is provided for posterity, but should not be run unless a resulting '<timestamp>-umax2012-fact-bln.csv' file is not present as it the runtime is very high.

This is an issue with the bi database in 2012, the views are extremely unoptimized.

In [14]:
# WARNING QUERY TAKES ~12-15 MINUTES TO RUN!!! RECOMMEND USING CSV FLAT FILE!!!
# Querying the 2012 bi database for kgals of consumption by contract.
fact_name = 'bln'
bi_query = '''
    SELECT DocumentDateKey
        , Contract_key
        , Premises_Key
        , Customer_Key
        , Service_Key
        , sum(linequantity) AS kgals
    FROM vw_BillLine
    WHERE billingclassification = 'water'
    AND unit = 'kgal'
    AND LEFT(CONVERT(VARCHAR(8), DOCUMENTDATE_KEY), 4) = '2023'
    GROUP BY DocumentDate_key
        , Contract_Key
        , Premises_Key
        , Customer_Key
        , Service_Key
'''

df = umax2012.query_db('umax_bi_ext_prod', bi_query, output='DataFrame')

TypeError: query_db() got an unexpected keyword argument 'output'

In [11]:
# write out file to csv for future use in the 'data/raw' directory.
current_timestamp = datetime.datetime.now()
formatted_timestamp = current_timestamp.strftime("%Y%m%d-%H%M%S")
df.to_csv(f'../data/raw/{formatted_timestamp}-umax2012-fact-{fact_name}.csv', index=False)

In [12]:
# remove the reference to the dataframe object and trigger garbage collection to reclaim memory
del df
gc.collect()

1776

This marks the end of the code that should only be run if the flat file is missing.  

---

## Collecting metadata
In this section we will extract metadata from both the 2012 and 365 databases. This metadata will include the following:
- Contract data
- Premises data
- Customer data
- Service data

Each of these datasets will be dimensional to our fact table, observe the relationship diagram below

```plaintext
┌───────────────┐      1:n┌─────────────┐
│FACT: BillLine │    ┌────┤DIM: Customer│
└───────▲───────┘    │    └─────────────┘
        │            │
        │            │
┌───────┴───────┐    │ 1:n┌─────────────┐
│ DIM: Contract ◄────┼────┤DIM: Premises│
└───────────────┘    │    └─────────────┘
                     │
                     │
                     │ 1:n┌─────────────┐
                     └────┤DIM: Service │
                          └─────────────┘
```

### Dimensional dataset workflows - 2012

In [11]:
fact_df = pd.read_csv('../data/raw/20240206-123237-umax2012-fact-bln.csv')

### Contract

In [95]:
# Creating the Contract dimensional dataset from 2012
dim_name = 'ctr'

query ='''
SELECT Contract_key
	, Product
	, AGR_StartDate
	, AGR_EndDate
	, Premises_key
	, Customer_key
	, Service_key
FROM vw_Contract 
WHERE billingclassification = 'Water'
AND StatementBasis LIKE 'consumption%'
'''

ctr_df = umax2012.query_db('UMAX_BI_EXT_PROD', query, output='DataFrame')

In [97]:
# write to flat file for storage.
current_timestamp = datetime.datetime.now()
formatted_timestamp = current_timestamp.strftime("%Y%m%d-%H%M%S")
ctr_df.to_csv(f'../data/raw/{formatted_timestamp}-umax2012-dim-{dim_name}.csv', index=False)

#### Premises

In [99]:
# Creating the Premises dimensional dataset from 2012
dim_name = 'prm'
query = '''
SELECT Premises_Key
	, PREM_Type
	, CityLimit
	, ExtraTerritorialJurisdictionalBoundary
FROM vw_Premises
WHERE Premises_Key IN (
    SELECT Premises_Key
    FROM vw_Contract
    WHERE BillingClassification = 'Water'
);
'''

prm_df = umax2012.query_db('UMAX_BI_EXT_PROD', query, output='DataFrame')

In [100]:
# write to flat file for storage.
current_timestamp = datetime.datetime.now()
formatted_timestamp = current_timestamp.strftime("%Y%m%d-%H%M%S")
prm_df.to_csv(f'../data/raw/{formatted_timestamp}-umax2012-dim-{dim_name}.csv', index=False)

### Customer

In [101]:
# Creating the Customer dimensional dataset from 2012
dim_name = 'cus'
query = '''
SELECT Customer_Key
	, CustomerGroup
FROM vw_Customer
WHERE Customer_Key IN (
    SELECT Customer_Key
    FROM vw_Contract
    WHERE BillingClassification = 'Water'
);
'''

cus_df = umax2012.query_db('UMAX_BI_EXT_PROD', query, output='DataFrame')

In [102]:
# write to flat file for storage.
current_timestamp = datetime.datetime.now()
formatted_timestamp = current_timestamp.strftime("%Y%m%d-%H%M%S")
cus_df.to_csv(f'../data/raw/{formatted_timestamp}-umax2012-dim-{dim_name}.csv', index=False)

### Service

In [169]:
# Creating the Service dimensional dataset from 2012
dim_name = 'srv'
query = '''
SELECT Service_Key
	, SERV_Type
	, SERV_County
FROM vw_Service
WHERE SERV_DeliveryType = 'Water';
'''
srv_df = umax2012.query_db('UMAX_BI_EXT_PROD', query, output='DataFrame')

In [170]:
# write to flat file for storage.
current_timestamp = datetime.datetime.now()
formatted_timestamp = current_timestamp.strftime("%Y%m%d-%H%M%S")
srv_df.to_csv(f'../data/raw/{formatted_timestamp}-umax2012-dim-{dim_name}.csv', index=False)

## BYOD data
First, we must instantiate the engine and session for our queries. This currently requires MFA.

In [110]:
# Dependencies
from sqlalchemy import text
from sqlalchemy.orm import sessionmaker
from src.pymax import umax365

In [107]:
engine = umax365.create_byod_engine()

Session = sessionmaker(bind=engine)
session = Session()

## Constructing the 365 BYOD fact dataset

In [111]:
# Due to drastically increased performance of the BYOD database, we will simply join all relevant attributes in a single query.
fact_name = 'bln'
bln_query = '''
    SELECT bln.DOCUMENTDATE
        , bln.CONTRACT_KEY
        , ctr.PRODUCT
        , prm.EXTPROPERTY_VALUE as PREMTYPE
        , cus.CUSTOMERGROUP
        , srv.SERV_COUNTY
        , SUM(bln.LINEQUANTITY) AS kgals
    FROM LNA_BIV_BillLine AS bln
    JOIN LNA_BIV_Contract AS ctr
    ON bln.CONTRACT_KEY = ctr.CONTRACT_KEY
    JOIN LNA_BIV_ExtProperty AS prm
    ON bln.PREMISES_KEY = prm.RELATEDTABLE_KEY 
        AND RELATEDTABLE_NAME = 'LNA_BIV_Premises' 
        AND EXTPROPERTY = 'PREMTYPE'
    JOIN LNA_BIV_Customer AS cus
    ON bln.CUSTOMER_KEY = cus.CUSTOMER_KEY
    JOIN LNA_BIV_Service AS srv
    ON bln.SERVICE_KEY = srv.SERVICE_KEY
    
    WHERE bln.BILLINGCLASSIFICATION = 'Water'
    AND bln.UNIT = 'kgal'
    
    GROUP BY bln.DOCUMENTDATE
        , bln.CONTRACT_KEY
        , ctr.PRODUCT
        , prm.EXTPROPERTY_VALUE
        , cus.CUSTOMERGROUP
        , srv.SERV_COUNTY
    
    ORDER BY bln.DOCUMENTDATE
'''

bln_result = session.execute(text(bln_query))
bln_df = pd.DataFrame(bln_result.fetchall(), columns=bln_result.keys())

# write out file to csv for future use in the 'data/raw' directory.
current_timestamp = datetime.datetime.now()
formatted_timestamp = current_timestamp.strftime("%Y%m%d-%H%M%S")
bln_df.to_csv(f'../data/raw/{formatted_timestamp}-umax365-fact-{fact_name}.csv', index=False)

In [113]:
bln_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 262007 entries, 0 to 262006
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   DOCUMENTDATE   262007 non-null  datetime64[ns]
 1   CONTRACT_KEY   262007 non-null  int64         
 2   PRODUCT        262007 non-null  object        
 3   PREMTYPE       262007 non-null  object        
 4   CUSTOMERGROUP  262007 non-null  object        
 5   SERV_COUNTY    262007 non-null  object        
 6   kgals          262007 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(5)
memory usage: 14.0+ MB


In [114]:
# We must now close the result and session to properly manage the connection pool and avoid memory leaks.
bln_result.close()
session.close()

## Building the final dataset

Now that we have extracted data from both databases, we will now need to do the following steps:

1. Join the 2012 data to match the format of our BYOD query
2. Concatenate the BYOD dataset to our resulting 2012 dataset
3. Introduce the required logic to derive customer classes and summarize the data

### Joining the 2012 results into a combined dataset

In [183]:
dtypes = {'contract': object, 'documentdate_key': object, 'kgal': float}
df = pd.read_csv('../data/raw/20240206-123237-umax2012-fact-bln.csv', names=dtypes.keys(), dtype = dtypes, header=0)

In [184]:
df.head()

,contract,documentdate_key,kgal
0,5637634178,20230531,4.0
1,5637875138,20230531,2.0
2,5637740937,20230112,2.0
3,5637768335,20230602,2.0
4,5637720913,20230112,2.0


In [185]:
df['doc_date'] = pd.to_datetime(df.documentdate_key, format='%Y%m%d')

In [186]:
df.head()

,contract,documentdate_key,kgal,doc_date
0,5637634178,20230531,4.0,2023-05-31
1,5637875138,20230531,2.0,2023-05-31
2,5637740937,20230112,2.0,2023-01-12
3,5637768335,20230602,2.0,2023-06-02
4,5637720913,20230112,2.0,2023-01-12


In [187]:
column = df.pop('doc_date')
df.insert(1, column.name, column)
df.head()

,contract,doc_date,documentdate_key,kgal
0,5637634178,2023-05-31,20230531,4.0
1,5637875138,2023-05-31,20230531,2.0
2,5637740937,2023-01-12,20230112,2.0
3,5637768335,2023-06-02,20230602,2.0
4,5637720913,2023-01-12,20230112,2.0


In [188]:
df.drop('documentdate_key', axis=1, inplace=True)

In [189]:
df.head()

,contract,doc_date,kgal
0,5637634178,2023-05-31,4.0
1,5637875138,2023-05-31,2.0
2,5637740937,2023-01-12,2.0
3,5637768335,2023-06-02,2.0
4,5637720913,2023-01-12,2.0


In [190]:
dtypes = {'contract': object, 'product': str, 'startdate': object, 'enddate': object, 'premises': object, 'customer': object, 'service': object}
dim_ctr = pd.read_csv('../data/raw/20240212-091912-umax2012-dim-ctr.csv', names=dtypes.keys(), dtype=dtypes, header=0)

In [191]:
dim_ctr.head()

,contract,product,startdate,enddate,premises,customer,service
0,5637152847,Water,2014-07-17,NaN,5637148436,5637150306,5637240266
1,5637152868,Water,2016-12-28,NaN,5637187276,5637201747,5637267598
2,5637152917,Water,2014-09-19,2021-12-16,5637160638,5637165412,5637227496
3,5637152958,Water,2012-03-01,NaN,5637160127,5637166631,5637239237
4,5637152976,Water,2015-02-27,NaN,5637170741,5637183317,5637286959


In [192]:
merged_df = pd.merge(df, dim_ctr, on='contract', how='left')

In [193]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 477611 entries, 0 to 477610
Data columns (total 9 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   contract   477611 non-null  object        
 1   doc_date   477611 non-null  datetime64[ns]
 2   kgal       477611 non-null  float64       
 3   product    477602 non-null  object        
 4   startdate  477602 non-null  object        
 5   enddate    31277 non-null   object        
 6   premises   477602 non-null  object        
 7   customer   477602 non-null  object        
 8   service    477602 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(7)
memory usage: 32.8+ MB


In [194]:
del df, dim_ctr

In [195]:
dtypes = {'premises': object, 'prem_type': str, 'city_limit': str, 'etj': str}
df = pd.read_csv('../data/raw/20240212-092027-umax2012-dim-prm.csv', names=dtypes.keys(), dtype=dtypes, header=0)
df.head()

,premises,prem_type,city_limit,etj
0,5637144576,Single family home,Other,GEORGETOWN
1,5637144578,Single family home,Other,LIBERTY HILL
2,5637144580,Single family home,Other,GEORGETOWN
3,5637144587,Single family home,GEORGETOWN,GEORGETOWN
4,5637144589,Single family home,GEORGETOWN,GEORGETOWN


In [196]:
merged_df = pd.merge(merged_df, df, on='premises', how='left')

In [197]:
del df

In [198]:
dtypes = {'customer': object, 'customer_group': str}
df = pd.read_csv('../data/raw/20240212-092129-umax2012-dim-cus.csv', names=dtypes.keys(), dtype=dtypes, header=0)
df.head()

,customer,customer_group
0,5637145389,Residential
1,5637145399,Residential
2,5637145411,Residential
3,5637145431,Residential
4,5637145438,Residential


In [199]:
merged_df = pd.merge(merged_df, df, on='customer', how='left')
merged_df.head()

,contract,doc_date,kgal,product,startdate,enddate,premises,customer,service,prem_type,city_limit,etj,customer_group
0,5637634178,2023-05-31,4.0,Water,2020-12-11,NaN,5637222254,5637268264,5637344598,Single family home,Other,NaN,Residential
1,5637875138,2023-05-31,2.0,Water,2023-05-10,NaN,5637242547,5637336821,5637382136,Single family home,Other,LIBERTY HILL,Residential
2,5637740937,2023-01-12,2.0,Water,2022-01-31,NaN,5637185481,5637298709,5637166911,Single family home,GEORGETOWN,GEORGETOWN,Residential
3,5637768335,2023-06-02,2.0,Water,2022-05-10,NaN,5637151807,5637306866,5637226741,Single family home,GEORGETOWN,GEORGETOWN,Residential
4,5637720913,2023-01-12,2.0,Water,2021-11-01,NaN,5637152481,5637291986,5637278097,Single family home,GEORGETOWN,GEORGETOWN,Residential


In [200]:
del df

In [201]:
dtypes = {'service': object, 'serv_type': str, 'serv_county': str}
df = pd.read_csv('../data/raw/20240212-122134-umax2012-dim-srv.csv', names=dtypes.keys(), dtype=dtypes, header=0)
df.head()

,service,serv_type,serv_county
0,5637144586,Water,Williamson County
1,5637144597,Water,Williamson County
2,5637144630,Irrigation,Williamson County
3,5637144632,Water,Williamson County
4,5637144670,Water,Williamson County


In [202]:
merged_df = pd.merge(merged_df, df, on='service', how='left')
merged_df.head()

,contract,doc_date,kgal,product,startdate,enddate,premises,customer,service,prem_type,city_limit,etj,customer_group,serv_type,serv_county
0,5637634178,2023-05-31,4.0,Water,2020-12-11,NaN,5637222254,5637268264,5637344598,Single family home,Other,NaN,Residential,Water,Williamson County
1,5637875138,2023-05-31,2.0,Water,2023-05-10,NaN,5637242547,5637336821,5637382136,Single family home,Other,LIBERTY HILL,Residential,Water,Williamson County
2,5637740937,2023-01-12,2.0,Water,2022-01-31,NaN,5637185481,5637298709,5637166911,Single family home,GEORGETOWN,GEORGETOWN,Residential,Water,Williamson County
3,5637768335,2023-06-02,2.0,Water,2022-05-10,NaN,5637151807,5637306866,5637226741,Single family home,GEORGETOWN,GEORGETOWN,Residential,Water,Williamson County
4,5637720913,2023-01-12,2.0,Water,2021-11-01,NaN,5637152481,5637291986,5637278097,Single family home,GEORGETOWN,GEORGETOWN,Residential,Water,Williamson County


In [203]:
del df

In [204]:
drop_cols = ['service', 'customer', 'premises']
merged_df.drop(drop_cols, axis=1, inplace=True)

In [205]:
merged_df.head()

,contract,doc_date,kgal,product,startdate,enddate,prem_type,city_limit,etj,customer_group,serv_type,serv_county
0,5637634178,2023-05-31,4.0,Water,2020-12-11,NaN,Single family home,Other,NaN,Residential,Water,Williamson County
1,5637875138,2023-05-31,2.0,Water,2023-05-10,NaN,Single family home,Other,LIBERTY HILL,Residential,Water,Williamson County
2,5637740937,2023-01-12,2.0,Water,2022-01-31,NaN,Single family home,GEORGETOWN,GEORGETOWN,Residential,Water,Williamson County
3,5637768335,2023-06-02,2.0,Water,2022-05-10,NaN,Single family home,GEORGETOWN,GEORGETOWN,Residential,Water,Williamson County
4,5637720913,2023-01-12,2.0,Water,2021-11-01,NaN,Single family home,GEORGETOWN,GEORGETOWN,Residential,Water,Williamson County


In [206]:
merged_df.contract.info()

<class 'pandas.core.series.Series'>
RangeIndex: 477611 entries, 0 to 477610
Series name: contract
Non-Null Count   Dtype 
--------------   ----- 
477611 non-null  object
dtypes: object(1)
memory usage: 3.6+ MB


In [207]:
merged_df.contract.nunique()

62865

In [208]:
merged_df.drop(['startdate', 'enddate', 'city_limit', 'etj', 'serv_type'], axis=1, inplace=True)

In [209]:
merged_df.head()

,contract,doc_date,kgal,product,prem_type,customer_group,serv_county
0,5637634178,2023-05-31,4.0,Water,Single family home,Residential,Williamson County
1,5637875138,2023-05-31,2.0,Water,Single family home,Residential,Williamson County
2,5637740937,2023-01-12,2.0,Water,Single family home,Residential,Williamson County
3,5637768335,2023-06-02,2.0,Water,Single family home,Residential,Williamson County
4,5637720913,2023-01-12,2.0,Water,Single family home,Residential,Williamson County


In [210]:
# moving columns to match destination format
columns_to_pop = ['contract', 'kgal']
new_index_pos = [1, 6]
for column_name, index_pos in zip(columns_to_pop, new_index_pos):
    column = merged_df.pop(column_name)
    merged_df.insert(index_pos, column_name, column)

In [247]:
merged_df.head()

,doc_date,contract,product,prem_type,customer_group,serv_county,kgal
0,2023-05-31,5637634178,Water,Single family home,Residential,Williamson County,4.0
1,2023-05-31,5637875138,Water,Single family home,Residential,Williamson County,2.0
2,2023-01-12,5637740937,Water,Single family home,Residential,Williamson County,2.0
3,2023-06-02,5637768335,Water,Single family home,Residential,Williamson County,2.0
4,2023-01-12,5637720913,Water,Single family home,Residential,Williamson County,2.0


## Concatenating the 2012 and 365 datasets

In [248]:
bln_df.head()

,doc_date,contract,product,prem_type,customer_group,serv_county,kgal
0,2023-10-01,5637161024,Water,Single family,Residential,WILLIAMSON,103.000000
1,2023-10-01,5637171867,Water,Single family,Residential,WILLIAMSON,13.000000
2,2023-10-01,5637192883,Water,Single family,Residential,WILLIAMSON,3.000000
3,2023-10-01,5637232787,Water,Single family,Residential,WILLIAMSON,1.000000
4,2023-10-01,5637261933,Water,Single family,Residential,WILLIAMSON,4.000000


In [249]:
bln_rename = {
    'DOCUMENTDATE': 'doc_date'
    , 'CONTRACT_KEY': 'contract'
    , 'PRODUCT': 'product'
    , 'SERVICE': 'service'
    , 'PREMTYPE': 'prem_type'
    , 'CUSTOMERGROUP': 'customer_group'
    , 'SERV_COUNTY': 'serv_county'
    , 'kgals': 'kgal'
}   
bln_df.rename(columns=bln_rename, inplace=True)

In [250]:
bln_df.head()

,doc_date,contract,product,prem_type,customer_group,serv_county,kgal
0,2023-10-01,5637161024,Water,Single family,Residential,WILLIAMSON,103.000000
1,2023-10-01,5637171867,Water,Single family,Residential,WILLIAMSON,13.000000
2,2023-10-01,5637192883,Water,Single family,Residential,WILLIAMSON,3.000000
3,2023-10-01,5637232787,Water,Single family,Residential,WILLIAMSON,1.000000
4,2023-10-01,5637261933,Water,Single family,Residential,WILLIAMSON,4.000000


In [251]:
cat_df = pd.concat([merged_df, bln_df])

In [252]:
cat_df.head()

,doc_date,contract,product,prem_type,customer_group,serv_county,kgal
0,2023-05-31,5637634178,Water,Single family home,Residential,Williamson County,4.0
1,2023-05-31,5637875138,Water,Single family home,Residential,Williamson County,2.0
2,2023-01-12,5637740937,Water,Single family home,Residential,Williamson County,2.0
3,2023-06-02,5637768335,Water,Single family home,Residential,Williamson County,2.0
4,2023-01-12,5637720913,Water,Single family home,Residential,Williamson County,2.0


In [253]:
df = cat_df.copy()
del cat_df

### Cleaning dataset

In [254]:
df['product'].value_counts()

product
Water                 729060
Irrigation              7585
Multi-family water      1074
Hydrant                  940
Multi_Family_Water       924
LH wholesale               9
Florence wholesale         7
Florence_Wholesale         5
RV_Park_Water              4
LH_Wholesale               1
Name: count, dtype: int64

In [255]:
prod_dicts = [
    {'Water': 'water'}
    , {'Irrigation': 'irrigation'}
    , {'Multi-family water': 'multifamily'}
    , {'Multi_Family_Water': 'multifamily'}
    , {'LH wholesale': 'liberty_hill_wholesale'}
    , {'LH_Wholesale': 'liberty_hill_wholesale'}
    , {'Florence wholesale': 'florence_wholesale'}
    , {'Florence_Wholesale': 'florence_wholesale'}
    , {'RV_Park_Water': 'rv_park'}
    , {'Hydrant': 'hydrant'}
]
for element in prod_dicts:
    for key, value in element.items():
        df['product'] = df['product'].replace(key, value)
        
df['product'].value_counts()

product
water                     729060
irrigation                  7585
multifamily                 1998
hydrant                      940
florence_wholesale            12
liberty_hill_wholesale        10
rv_park                        4
Name: count, dtype: int64

In [256]:
df.prem_type.value_counts()

prem_type
Single family home    451186
Single family         243938
Non-residential        24501
Duplex                  7797
Fourplex                4563
Complex unit            2442
Complex                 2035
Complex Unit            1284
To be determined         862
Triplex                   63
Name: count, dtype: int64

In [257]:
prem_dicts = [
    {'Single family home': 'single_family'}
    , {'Single family': 'single_family'}
    , {'Complex unit': 'complex_unit'}
    , {'Complex Unit': 'complex_unit'}
    , {'Non-residential': 'non_residential'}
    , {'Duplex': 'duplex'}
    , {'Triplex': 'triplex'}
    , {'Fourplex': 'fourplex'}
    , {'To be determined': 'tbd'}
    , {'Complex': 'complex'}
]
for element in prem_dicts:
    for key, value in element.items():
        df['prem_type'] = df['prem_type'].replace(key, value)
        
df['prem_type'].value_counts()

prem_type
single_family      695124
non_residential     24501
duplex               7797
fourplex             4563
complex_unit         3726
complex              2035
tbd                   862
triplex                63
Name: count, dtype: int64

In [242]:
df['customer_group'].value_counts()

customer_group
Residential           711434
Commercial             24994
City of Georgetown      1806
Government               601
School                   437
Industrial               337
Name: count, dtype: int64

In [259]:
df['customer_group'] = df['customer_group'].str.lower()
df['customer_group'].value_counts()

customer_group
residential           711434
commercial             24994
city_of_georgetown      1806
government               601
school                   437
industrial               337
Name: count, dtype: int64

In [260]:
df['customer_group'] = df['customer_group'].str.replace(' ', '_')
df['customer_group'].value_counts()

customer_group
residential           711434
commercial             24994
city_of_georgetown      1806
government               601
school                   437
industrial               337
Name: count, dtype: int64

In [261]:
df['serv_county'] = df['serv_county'].str.lower()
df['serv_county'] = df['serv_county'].str.replace(' ', '_')
df['serv_county'].value_counts()

serv_county
williamson_county    475677
williamson           260785
bell                   2902
burnet                  228
Name: count, dtype: int64

In [262]:
df['serv_county'] = df['serv_county'].replace('williamson_county', 'williamson')
df['serv_county'].value_counts()

serv_county
williamson    736462
bell            2902
burnet           228
Name: count, dtype: int64

In [265]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 739618 entries, 0 to 262006
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   doc_date        739618 non-null  datetime64[ns]
 1   contract        739618 non-null  object        
 2   product         739609 non-null  object        
 3   prem_type       738671 non-null  object        
 4   customer_group  739609 non-null  object        
 5   serv_county     739592 non-null  object        
 6   kgal            739618 non-null  object        
dtypes: datetime64[ns](1), object(6)
memory usage: 45.1+ MB


### Defining the derived_class function

In [310]:
def assign_derived_class(row):
    if (row['prem_type'] in ['single_family', 'tbd'] 
        and row['product'] in ['irrigation', 'water'] 
        and row['customer_group'] == 'residential'):
        return 'single_family'
    elif ((row['prem_type'] in ['complex', 'complex_unit'] 
           and row['product'] in ['multifamily', 'water'] 
           and row['customer_group'] in ['commercial', 'residential'])
          or (row['prem_type'] in ['duplex', 'triplex', 'fourplex', 'single_family'] 
              and row['product'] in ['multifamily', 'water'] 
              and row['customer_group'] == 'residential')
          or (row['prem_type'] == 'non_residential' 
              and row['product'] == 'rv_park')):
        return 'multifamily'
    elif ((row['prem_type'] in ['single_family', 'tbd'] 
           and row['product'] in ['irrigation', 'water'] 
           and row['customer_group'] in ['commercial', 'industrial'])
          or (row['prem_type'] in ['complex', 'complex_unit'] 
              and row['product'] == 'irrigation')
          or (row['prem_type'] == 'non_residential' 
              and row['product'] in ['multifamily', 'water', 'irrigation'] 
              and row['customer_group'] in ['commercial', 'residential'])
          or (row['product'] == 'hydrant')):
        return 'commercial'
    elif (row['prem_type'] == 'non_residential' 
          and row['product'] in ['irrigation', 'water'] 
          and row['customer_group'] == 'industrial'):
        return 'industrial'
    elif (row['product'] not in ['georgetown_cc', 'non_potable_water'] 
          and row['customer_group'] in ['government', 'city_of_georgetown', 'school']):
        return 'institutional'
    elif (row['prem_type'] == 'non_residential' 
          and row['product'] in ['georgetown_cc', 'non_potable_water'] 
          and row['customer_group'] in ['commercial', 'city_of_georgetown']):
        return 'reuse'
    else:
        return 'unknown'

In [275]:
test_df = df.sample(10).copy()
test_df['derived_class'] = test_df.apply(assign_derived_class, axis=1)
test_df

,doc_date,contract,product,prem_type,customer_group,serv_county,kgal,derived_class
113696,2023-11-28,5637233725,water,single_family,residential,williamson,9.000000,single_family
468635,2023-01-05,5637661012,water,single_family,residential,williamson,4.0,single_family
43871,2023-09-04,5637598092,water,single_family,residential,williamson,1.0,single_family
191000,2023-05-31,5637289690,water,single_family,residential,williamson,5.0,single_family
84422,2023-02-28,5637181587,irrigation,non_residential,commercial,williamson,206.0,commercial
56441,2023-10-30,5637377321,water,single_family,residential,williamson,2.000000,single_family
135299,2023-12-06,5637462512,water,single_family,residential,williamson,1.000000,single_family
109721,2023-06-12,5637333237,water,single_family,residential,williamson,16.0,single_family
179217,2023-03-30,5637762336,water,single_family,residential,williamson,16.0,single_family
375610,2023-03-15,5637358362,water,single_family,residential,williamson,1.0,single_family


In [311]:
df['derived_class'] = df.apply(assign_derived_class, axis=1)
df.head()

,doc_date,contract,product,prem_type,customer_group,serv_county,kgal,derived_class
0,2023-05-31,5637634178,water,single_family,residential,williamson,4.0,single_family
1,2023-05-31,5637875138,water,single_family,residential,williamson,2.0,single_family
2,2023-01-12,5637740937,water,single_family,residential,williamson,2.0,single_family
3,2023-06-02,5637768335,water,single_family,residential,williamson,2.0,single_family
4,2023-01-12,5637720913,water,single_family,residential,williamson,2.0,single_family


In [312]:
df.loc[(df['derived_class'] == 'unknown')]

,doc_date,contract,product,prem_type,customer_group,serv_county,kgal,derived_class
35731,2023-06-23,5637647415,NaN,NaN,NaN,NaN,11.0,unknown
162182,2023-04-27,5637647415,NaN,NaN,NaN,NaN,7.0,unknown
232299,2023-03-01,5637647415,NaN,NaN,NaN,NaN,9.0,unknown
269383,2023-05-31,5637647415,NaN,NaN,NaN,NaN,11.0,unknown
275274,2023-09-21,5637647415,NaN,NaN,NaN,NaN,9.0,unknown
303943,2023-07-25,5637647415,NaN,NaN,NaN,NaN,12.0,unknown
308582,2023-08-23,5637647415,NaN,NaN,NaN,NaN,13.0,unknown
369300,2023-01-26,5637647415,NaN,NaN,NaN,NaN,11.0,unknown
425282,2023-03-29,5637647415,NaN,NaN,NaN,NaN,7.0,unknown


In [313]:
df.kgal = pd.to_numeric(df.kgal)

In [317]:
df.groupby([df['doc_date'].dt.year, 'derived_class', 'serv_county'])['kgal'].sum().reset_index().to_csv('../data/processed/kgal-by-class.csv')